In [70]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

### Acquire the train.csv

In [2]:
# Loade the csv files
train = pd.read_csv("train.csv")

# Inspect the shapes
train.shape

(101230332, 10)

In [3]:
# Take a quck peek of the dataframe
train.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


In [4]:
# Summarize the train dataset
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101230332 entries, 0 to 101230331
Data columns (total 10 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   row_id                          int64  
 1   timestamp                       int64  
 2   user_id                         int64  
 3   content_id                      int64  
 4   content_type_id                 int64  
 5   task_container_id               int64  
 6   user_answer                     int64  
 7   answered_correctly              int64  
 8   prior_question_elapsed_time     float64
 9   prior_question_had_explanation  object 
dtypes: float64(1), int64(8), object(1)
memory usage: 7.5+ GB


**Takeaways**
- The train dataset has more than 100 millions records. I only need to small portion to calculate the statistics of the past.<br>
    -**I will take the first 1000 user ids.**

In [8]:
# Extract the first 1000 user ids

user_ids1000 = train.user_id.value_counts().sort_index().iloc[0: 1000]
user_ids1000 = user_ids1000.index.to_list()

# Sanity check
print(len(user_ids1000))

# Print the first 5 user ids
user_ids1000[:5]

1000


[115, 124, 2746, 5382, 8623]

In [10]:
# Set the user_id to the index

train = train.set_index("user_id")
train.head()

,row_id,timestamp,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
user_id,,,,,,,,,
115,0,0,5692,0,1,3,1,NaN,NaN
115,1,56943,5716,0,2,2,1,37000.0,False
115,2,118363,128,0,0,0,1,55000.0,False
115,3,131167,7860,0,3,0,1,19000.0,False
115,4,137965,7922,0,4,1,1,11000.0,False


In [11]:
# Subtracts the records for the first 1000 users
train1000 = train.loc[user_ids1000]

# Take a peek at the new dataset
train1000.head()

,row_id,timestamp,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
user_id,,,,,,,,,
115,0,0,5692,0,1,3,1,NaN,NaN
115,1,56943,5716,0,2,2,1,37000.0,False
115,2,118363,128,0,0,0,1,55000.0,False
115,3,131167,7860,0,3,0,1,19000.0,False
115,4,137965,7922,0,4,1,1,11000.0,False


In [12]:
# Sanity check
train1000.tail() # should match the tail of user_ids1000

,row_id,timestamp,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
user_id,,,,,,,,,
5577726,296610,625311764,5328,0,7,0,0,20000.0,False
5577726,296611,625327721,4303,0,8,3,0,10000.0,True
5577726,296612,625344560,4445,0,9,0,1,6000.0,True
5577726,296613,625367161,5206,0,10,1,0,6000.0,True
5577726,296614,625378759,3928,0,11,0,0,13000.0,True


In [14]:
# Print the tail of the user_ids1000
user_ids1000[999]

5577726

In [60]:
# Read the train1000.csv
train1000 = pd.read_csv("train1000.csv", index_col=0)

In [61]:
# Quick summarize the train1000
train1000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296615 entries, 115 to 5577726
Data columns (total 9 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   row_id                          296615 non-null  int64  
 1   timestamp                       296615 non-null  int64  
 2   content_id                      296615 non-null  int64  
 3   content_type_id                 296615 non-null  int64  
 4   task_container_id               296615 non-null  int64  
 5   user_answer                     296615 non-null  int64  
 6   answered_correctly              296615 non-null  int64  
 7   prior_question_elapsed_time     289945 non-null  float64
 8   prior_question_had_explanation  295617 non-null  object 
dtypes: float64(1), int64(7), object(1)
memory usage: 22.6+ MB


In [62]:
# Count how many missing values in the prior_question_elapsed_time

mask = train1000.prior_question_elapsed_time.isnull()
mask.sum()

6670

In [63]:
# Count how many missing values in the prior_question_had_explanation

mask = train1000.prior_question_had_explanation.isnull()
mask.sum()

998

**Takeaways**
- The memoery usage of the first 1000 users are only 23 MB
- Missing values are found in columns:
    - prior_question_elapsed_time: 6670 nulls
    - prior_question_had_explanantion: 998 nulls

### Handling the null values

In [64]:
# The values of the two prior columns will always be 0 when the user first interacted with the system
# Take a first user 115 for example

train1000.loc[115].head()

,row_id,timestamp,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
user_id,,,,,,,,,
115,0,0,5692,0,1,3,1,NaN,NaN
115,1,56943,5716,0,2,2,1,37000.0,False
115,2,118363,128,0,0,0,1,55000.0,False
115,3,131167,7860,0,3,0,1,19000.0,False
115,4,137965,7922,0,4,1,1,11000.0,False


**Takeawasy**
- There are only 1000 users so the two prior columns should have at leat 1000 nulls.
- The prior_question_had_explanantion column has 998 nulls:
    - fill the nulls with 'False'
- I will further explore the missing values in the column prior_question_elapased_time.

### Fill the nulls in the prior_question_had_explanation with string 'False'

In [65]:
train1000.prior_question_had_explanation.fillna('False', inplace = True)
train1000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296615 entries, 115 to 5577726
Data columns (total 9 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   row_id                          296615 non-null  int64  
 1   timestamp                       296615 non-null  int64  
 2   content_id                      296615 non-null  int64  
 3   content_type_id                 296615 non-null  int64  
 4   task_container_id               296615 non-null  int64  
 5   user_answer                     296615 non-null  int64  
 6   answered_correctly              296615 non-null  int64  
 7   prior_question_elapsed_time     289945 non-null  float64
 8   prior_question_had_explanation  296615 non-null  object 
dtypes: float64(1), int64(7), object(1)
memory usage: 32.6+ MB


In [66]:
mask = train1000.prior_question_elapsed_time.isnull()
train1000[mask]

,row_id,timestamp,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
user_id,,,,,,,,,
115,0,0,5692,0,1,3,1,NaN,False
124,46,0,7900,0,0,0,1,NaN,False
2746,76,0,5273,0,0,1,0,NaN,False
2746,89,653762,6808,1,14,-1,-1,NaN,False
5382,96,0,5000,0,0,0,1,NaN,False
...,...,...,...,...,...,...,...,...,...
5566848,296498,162115714,32168,1,21,-1,-1,NaN,False
5568049,296509,0,6028,0,0,3,0,NaN,False
5568049,296537,7746409318,24250,1,20,-1,-1,NaN,False


**Takeaways**
- When the user studies the lecture, the prior_question_elapsed_time documented as NaN.
- I will fill the missing values in the columsn prior_question_elapsed_time with 0.

### Fill the nulls in the prior_question_elapsed_time with string 0

In [67]:
train1000.prior_question_elapsed_time.fillna(0, inplace = True)
train1000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296615 entries, 115 to 5577726
Data columns (total 9 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   row_id                          296615 non-null  int64  
 1   timestamp                       296615 non-null  int64  
 2   content_id                      296615 non-null  int64  
 3   content_type_id                 296615 non-null  int64  
 4   task_container_id               296615 non-null  int64  
 5   user_answer                     296615 non-null  int64  
 6   answered_correctly              296615 non-null  int64  
 7   prior_question_elapsed_time     296615 non-null  float64
 8   prior_question_had_explanation  296615 non-null  object 
dtypes: float64(1), int64(7), object(1)
memory usage: 32.6+ MB


**Takeawasy**
- All the null values have been addressed in the dataset. 

### Split the train1000 into train and test

In [69]:
# Double check the 1000 user_id list
print(len(user_ids1000))
user_ids1000[:5]

1000


[115, 124, 2746, 5382, 8623]

In [233]:
# Create the train and test dataframe

train = pd.DataFrame()
test = pd.DataFrame()

# Set up the train size
train_size = 0.8

for user_id in user_ids1000:
    if train1000.loc[[user_id]].shape[0] <=2:
        print(user_id)
        continue
    elif train1000.loc[[user_id]].shape[0] > 2: 
        df = train1000.loc[[user_id]]
        n = df.shape[0]
        test_start_index = round(train_size * n)
        df_train = df.iloc[:test_start_index]
        df_test = df.iloc[test_start_index:]     
        train = pd.concat([train, df_train])
        test = pd.concat([test, df_test])
    
# Print the shape of the original, train and test
train1000.shape, train.shape, test.shape

1880240
2148001


((296615, 9), (237322, 9), (59290, 9))

In [234]:
# Take a peek at the train dataset
train.head(2)

,row_id,timestamp,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
user_id,,,,,,,,,
115,0,0,5692,0,1,3,1,0.0,False
115,1,56943,5716,0,2,2,1,37000.0,False


In [235]:
train.tail(2)

,row_id,timestamp,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
user_id,,,,,,,,,
5577726,296611,625327721,4303,0,8,3,0,10000.0,True
5577726,296612,625344560,4445,0,9,0,1,6000.0,True


In [236]:
# Take a peek at the test
test.head(2)

,row_id,timestamp,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
user_id,,,,,,,,,
115,37,1415188,7882,0,37,2,1,20000.0,True
115,38,1468285,7962,0,38,3,1,17000.0,True


In [237]:
test.tail(2)

,row_id,timestamp,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
user_id,,,,,,,,,
5577726,296613,625367161,5206,0,10,1,0,6000.0,True
5577726,296614,625378759,3928,0,11,0,0,13000.0,True


### Count how many lectures each user has reviewed in the train

In [238]:
# Create a new columns for if the student review the lecture or not

train['reviewed_lecture'] = (train['answered_correctly'] == -1)
train

,row_id,timestamp,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,reviewed_lecture
user_id,,,,,,,,,,
115,0,0,5692,0,1,3,1,0.0,False,False
115,1,56943,5716,0,2,2,1,37000.0,False,False
115,2,118363,128,0,0,0,1,55000.0,False,False
115,3,131167,7860,0,3,0,1,19000.0,False,False
115,4,137965,7922,0,4,1,1,11000.0,False,False
...,...,...,...,...,...,...,...,...,...,...
5577726,296608,95083,5153,0,5,1,1,14000.0,False,False
5577726,296609,118668,5266,0,6,3,0,20000.0,False,False
5577726,296610,625311764,5328,0,7,0,0,20000.0,False,False


In [239]:
# Compute how many times each user reviewed the lecture

count_reviewed_lecture = train.groupby(train.index).reviewed_lecture.sum()
count_reviewed_lecture

user_id
115        0
124        0
2746       1
5382       1
8623       2
          ..
5555086    0
5566848    2
5568049    1
5571864    0
5577726    0
Name: reviewed_lecture, Length: 998, dtype: int64

**Takeaways**
- What if the user didn't review the lecture in the train dataset but reference the lecture in the test dataset? 
- For this qucik run though, I didn't use it as the new feature

### Drop the lecture rows in train and test

In [240]:
# Drop the lecture rows in train

mask = train['answered_correctly'] != -1
train = train[mask]

# Print the shape
train.shape

(232718, 10)

**Quick note**: the number of observations in the train drops from 237322 to 232718

In [241]:
# Drop the lecture rows in test

mask = test['answered_correctly'] != -1
test = test[mask]

# Print the shape
test.shape

(58222, 9)

**Quick note**: the number of observations in the test drops from 59290 to 58222.

In [242]:
# Reset the index in the train and test
train.reset_index(inplace=True)
train.head()

,user_id,row_id,timestamp,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,reviewed_lecture
0,115,0,0,5692,0,1,3,1,0.0,False,False
1,115,1,56943,5716,0,2,2,1,37000.0,False,False
2,115,2,118363,128,0,0,0,1,55000.0,False,False
3,115,3,131167,7860,0,3,0,1,19000.0,False,False
4,115,4,137965,7922,0,4,1,1,11000.0,False,False


In [243]:
# Reset the index in the train and test
test.reset_index(inplace=True)
test.head()

,user_id,row_id,timestamp,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,115,37,1415188,7882,0,37,2,1,20000.0,True
1,115,38,1468285,7962,0,38,3,1,17000.0,True
2,115,39,667861680,1278,0,39,3,1,21000.0,False
3,115,40,667971812,2065,0,40,2,1,17000.0,False
4,115,41,667971812,2064,0,40,1,1,17000.0,False


### Feature engineering - To measure the statistics of the whole content
- content_id
- task_container_id

In [244]:
# Compute the statistics of the content by the the content_id

content_stats = train.groupby('content_id').answered_correctly.agg(['mean', 
                                                                    'count', 
                                                                    'std', 
                                                                    'median', 
                                                                    'skew'])


# Rename the name of the columns

content_stats.columns = ['mean_content_accuracy', 
                         'question_content_asked', 
                         'std_content_accuracy', 
                         'median_content_accuracy', 
                         'skew_content_accuracy']

# Take a look at the dataframe
content_stats

,mean_content_accuracy,question_content_asked,std_content_accuracy,median_content_accuracy,skew_content_accuracy
content_id,,,,,
0,0.944444,18,0.235702,1.0,-4.242641
1,0.928571,14,0.267261,1.0,-3.741657
2,0.566372,113,0.497783,1.0,-0.271474
3,0.793103,58,0.408619,1.0,-1.485842
4,0.480519,77,0.502897,0.0,0.079539
...,...,...,...,...,...
13518,0.000000,1,NaN,0.0,NaN
13519,0.000000,1,NaN,0.0,NaN
13520,0.750000,4,0.500000,1.0,-2.000000


In [245]:
# Compute the statistics of the content by the the task_container_id

task_container_stats = train.groupby('task_container_id').answered_correctly.agg(['mean', 
                                                                                  'count', 
                                                                                  'std', 
                                                                                  'median', 
                                                                                  'skew'])


# Rename the name of the columns

task_container_stats.columns = ['mean_task_accuracy', 
                                'question_task_asked', 
                                'std_task_accuracy', 
                                'median_task_accuracy', 
                                'skew_task_accuracy']

# Take a look at the dataframe
task_container_stats

,mean_task_accuracy,question_task_asked,std_task_accuracy,median_task_accuracy,skew_task_accuracy
task_container_id,,,,,
0,0.696215,1004,0.460120,1.0,-0.854589
1,0.522908,1004,0.499724,1.0,-0.091867
2,0.429429,999,0.495243,0.0,0.285566
3,0.543912,1002,0.498317,1.0,-0.176595
4,0.493096,1738,0.500096,0.0,0.027644
...,...,...,...,...,...
4496,1.000000,1,NaN,1.0,NaN
4497,1.000000,1,NaN,1.0,NaN
4498,1.000000,1,NaN,1.0,NaN


### Feature Engineering: To measure the historical performance of each user
- mean user accuracy
- mean time to answer the question
- mean time to answer the prior question
- convert the column 'prior question had been explained nor not' to bool

In [246]:
# Take a peek at the train
train.head()

,user_id,row_id,timestamp,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,reviewed_lecture
0,115,0,0,5692,0,1,3,1,0.0,False,False
1,115,1,56943,5716,0,2,2,1,37000.0,False,False
2,115,2,118363,128,0,0,0,1,55000.0,False,False
3,115,3,131167,7860,0,3,0,1,19000.0,False,False
4,115,4,137965,7922,0,4,1,1,11000.0,False,False


In [250]:
# Compute the statistics of the answers given by every user

user_answer_stats = train.groupby("user_id").answered_correctly.agg(['mean', 
                                                                      'count', 
                                                                      'std', 
                                                                      'median', 
                                                                      'skew'])


# Rename the name of the columns

user_answer_stats.columns = ['mean_user_accuracy', 
                            'question_user_asked', 
                            'std_user_accuracy', 
                            'median_user_accuracy', 
                            'skew_user_accuracy']

# Take a look at the dataframe
user_answer_stats

,mean_user_accuracy,question_user_asked,std_user_accuracy,median_user_accuracy,skew_user_accuracy
user_id,,,,,
115,0.702703,37,0.463373,1.0,-0.924895
124,0.291667,24,0.464306,0.0,0.978979
2746,0.533333,15,0.516398,1.0,-0.148961
5382,0.693069,101,0.463521,1.0,-0.849885
8623,0.602273,88,0.492233,1.0,-0.425210
...,...,...,...,...,...
5555086,0.571429,14,0.513553,1.0,-0.324537
5566848,0.538462,26,0.508391,1.0,-0.163916
5568049,0.491803,61,0.504082,0.0,0.033624


In [251]:
# Compute the statistics of time used by each user

timestamp_stats = train.groupby("user_id").timestamp.agg(['mean', 
                                                          'count', 
                                                          'std', 
                                                          'median', 
                                                          'skew'])


# Rename the name of the columns

timestamp_stats.columns = ['mean_timestamp_accuracy', 
                           'question_timestamp_asked', 
                           'std_timestamp_accuracy', 
                           'median_timestamp_accuracy', 
                           'skew_timestamp_accuracy']

# Take a look at the dataframe
timestamp_stats

,mean_timestamp_accuracy,question_timestamp_asked,std_timestamp_accuracy,median_timestamp_accuracy,skew_timestamp_accuracy
user_id,,,,,
115,5.083378e+05,37,3.687448e+05,4.372720e+05,1.072164
124,3.183957e+05,24,1.681192e+05,3.305280e+05,-0.370039
2746,2.584739e+05,15,2.233817e+05,2.346050e+05,0.834430
5382,2.276186e+08,101,1.609300e+08,2.993556e+08,-0.415909
8623,2.569741e+08,88,1.640725e+08,3.202814e+08,-0.009533
...,...,...,...,...,...
5555086,7.458212e+05,14,5.197685e+05,8.824825e+05,-0.431008
5566848,5.989637e+07,26,7.548054e+07,2.909694e+06,0.625659
5568049,6.346203e+09,61,3.001274e+09,7.746523e+09,-1.705166


In [252]:
# Compute the statistics of prior question elapsed time used by each user

priortime_stats = train.groupby("user_id").prior_question_elapsed_time.agg(['mean', 
                                                                              'count', 
                                                                              'std', 
                                                                              'median', 
                                                                              'skew'])


# Rename the name of the columns

priortime_stats.columns = ['mean_priortime_accuracy', 
                           'question_priortime_asked', 
                           'std_priortime_accuracy', 
                           'median_priortime_accuracy', 
                           'skew_priortime_accuracy']

# Take a look at the dataframe
priortime_stats

,mean_priortime_accuracy,question_priortime_asked,std_priortime_accuracy,median_priortime_accuracy,skew_priortime_accuracy
user_id,,,,,
115,20135.135135,37,8310.509284,20000.0,1.796349
124,21041.541667,24,8581.065906,21333.0,-0.579390
2746,16866.666667,15,6104.643042,17000.0,-1.124964
5382,36910.891089,101,28382.071457,25000.0,2.471402
8623,27995.454545,88,19703.946212,21000.0,1.650412
...,...,...,...,...,...
5555086,27785.714286,14,13857.397755,26000.0,0.081749
5566848,29999.884615,26,12077.501951,32166.5,-0.136437
5568049,16364.639344,61,10334.513775,19000.0,0.290459


In [253]:
# Conver the dtype of prior_question_had_explanation to bool

bool_values = train.prior_question_had_explanation.apply(lambda i: True if i == 'True' else False)
train.prior_question_had_explanation = bool_values

# Print the data type of the train dataset
train.dtypes

user_id                             int64
row_id                              int64
timestamp                           int64
content_id                          int64
content_type_id                     int64
task_container_id                   int64
user_answer                         int64
answered_correctly                  int64
prior_question_elapsed_time       float64
prior_question_had_explanation       bool
reviewed_lecture                     bool
dtype: object

### Merge the new features with the train

In [254]:
# Merge with mean of the content accuracy
train = train.merge(content_stats[['mean_content_accuracy']], how='left', on='content_id')

# Drop the content_id and content_type_id columns
train.drop(columns=['content_id', 'content_type_id'], inplace=True)

# Print the new train
train

,user_id,row_id,timestamp,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,reviewed_lecture,mean_content_accuracy
0,115,0,0,1,3,1,0.0,False,False,0.702381
1,115,1,56943,2,2,1,37000.0,False,False,0.789474
2,115,2,118363,0,0,1,55000.0,False,False,0.979167
3,115,3,131167,3,0,1,19000.0,False,False,0.962963
4,115,4,137965,4,1,1,11000.0,False,False,0.977778
...,...,...,...,...,...,...,...,...,...,...
232713,5577726,296608,95083,5,1,1,14000.0,False,False,0.560000
232714,5577726,296609,118668,6,3,0,20000.0,False,False,0.573770
232715,5577726,296610,625311764,7,0,0,20000.0,False,False,0.750000
232716,5577726,296611,625327721,8,3,0,10000.0,False,False,0.541667


In [255]:
# Quick summarize the new train
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 232718 entries, 0 to 232717
Data columns (total 10 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   user_id                         232718 non-null  int64  
 1   row_id                          232718 non-null  int64  
 2   timestamp                       232718 non-null  int64  
 3   task_container_id               232718 non-null  int64  
 4   user_answer                     232718 non-null  int64  
 5   answered_correctly              232718 non-null  int64  
 6   prior_question_elapsed_time     232718 non-null  float64
 7   prior_question_had_explanation  232718 non-null  bool   
 8   reviewed_lecture                232718 non-null  bool   
 9   mean_content_accuracy           232718 non-null  float64
dtypes: bool(2), float64(2), int64(6)
memory usage: 16.4 MB


In [256]:
# Merge with mean of the task container accuracy
train = train.merge(task_container_stats[['mean_task_accuracy']], how='left', on='task_container_id')

# Drop the content_id and content_type_id columns
train.drop(columns=['task_container_id'], inplace=True)

# Print the new train
train

,user_id,row_id,timestamp,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,reviewed_lecture,mean_content_accuracy,mean_task_accuracy
0,115,0,0,3,1,0.0,False,False,0.702381,0.522908
1,115,1,56943,2,1,37000.0,False,False,0.789474,0.429429
2,115,2,118363,0,1,55000.0,False,False,0.979167,0.696215
3,115,3,131167,0,1,19000.0,False,False,0.962963,0.543912
4,115,4,137965,1,1,11000.0,False,False,0.977778,0.493096
...,...,...,...,...,...,...,...,...,...,...
232713,5577726,296608,95083,1,1,14000.0,False,False,0.560000,0.409666
232714,5577726,296609,118668,3,0,20000.0,False,False,0.573770,0.315236
232715,5577726,296610,625311764,0,0,20000.0,False,False,0.750000,0.523975
232716,5577726,296611,625327721,3,0,10000.0,False,False,0.541667,0.507859


In [257]:
# Quick summarize the new train
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 232718 entries, 0 to 232717
Data columns (total 10 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   user_id                         232718 non-null  int64  
 1   row_id                          232718 non-null  int64  
 2   timestamp                       232718 non-null  int64  
 3   user_answer                     232718 non-null  int64  
 4   answered_correctly              232718 non-null  int64  
 5   prior_question_elapsed_time     232718 non-null  float64
 6   prior_question_had_explanation  232718 non-null  bool   
 7   reviewed_lecture                232718 non-null  bool   
 8   mean_content_accuracy           232718 non-null  float64
 9   mean_task_accuracy              232718 non-null  float64
dtypes: bool(2), float64(3), int64(5)
memory usage: 16.4 MB


In [259]:
# Merge the mean_user_accuracy
train = train.merge(user_answer_stats[['mean_user_accuracy']], how="left", on="user_id")

# Drop the user_answer column
train.drop(columns="user_answer", inplace=True)

# Take a look at the new train
train

,user_id,row_id,timestamp,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,reviewed_lecture,mean_content_accuracy,mean_task_accuracy,mean_user_accuracy
0,115,0,0,1,0.0,False,False,0.702381,0.522908,0.702703
1,115,1,56943,1,37000.0,False,False,0.789474,0.429429,0.702703
2,115,2,118363,1,55000.0,False,False,0.979167,0.696215,0.702703
3,115,3,131167,1,19000.0,False,False,0.962963,0.543912,0.702703
4,115,4,137965,1,11000.0,False,False,0.977778,0.493096,0.702703
...,...,...,...,...,...,...,...,...,...,...
232713,5577726,296608,95083,1,14000.0,False,False,0.560000,0.409666,0.500000
232714,5577726,296609,118668,0,20000.0,False,False,0.573770,0.315236,0.500000
232715,5577726,296610,625311764,0,20000.0,False,False,0.750000,0.523975,0.500000
232716,5577726,296611,625327721,0,10000.0,False,False,0.541667,0.507859,0.500000


In [261]:
# Merge the mean_timestamp_accuracy
train = train.merge(timestamp_stats[['mean_timestamp_accuracy']], how="left", on="user_id")

# Drop the user_answer column
train.drop(columns="timestamp", inplace=True)

# Take a look at the new train
train

,user_id,row_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,reviewed_lecture,mean_content_accuracy,mean_task_accuracy,mean_user_accuracy,mean_timestamp_accuracy
0,115,0,1,0.0,False,False,0.702381,0.522908,0.702703,5.083378e+05
1,115,1,1,37000.0,False,False,0.789474,0.429429,0.702703,5.083378e+05
2,115,2,1,55000.0,False,False,0.979167,0.696215,0.702703,5.083378e+05
3,115,3,1,19000.0,False,False,0.962963,0.543912,0.702703,5.083378e+05
4,115,4,1,11000.0,False,False,0.977778,0.493096,0.702703,5.083378e+05
...,...,...,...,...,...,...,...,...,...,...
232713,5577726,296608,1,14000.0,False,False,0.560000,0.409666,0.500000,1.876389e+08
232714,5577726,296609,0,20000.0,False,False,0.573770,0.315236,0.500000,1.876389e+08
232715,5577726,296610,0,20000.0,False,False,0.750000,0.523975,0.500000,1.876389e+08
232716,5577726,296611,0,10000.0,False,False,0.541667,0.507859,0.500000,1.876389e+08


In [263]:
# Merge the mean_priortime_accuracy
train = train.merge(priortime_stats[['mean_priortime_accuracy']], how="left", on="user_id")

# Drop the user_answer column
train.drop(columns="prior_question_elapsed_time", inplace=True)

# Take a look at the new train
train

,user_id,row_id,answered_correctly,prior_question_had_explanation,reviewed_lecture,mean_content_accuracy,mean_task_accuracy,mean_user_accuracy,mean_timestamp_accuracy,mean_priortime_accuracy
0,115,0,1,False,False,0.702381,0.522908,0.702703,5.083378e+05,20135.135135
1,115,1,1,False,False,0.789474,0.429429,0.702703,5.083378e+05,20135.135135
2,115,2,1,False,False,0.979167,0.696215,0.702703,5.083378e+05,20135.135135
3,115,3,1,False,False,0.962963,0.543912,0.702703,5.083378e+05,20135.135135
4,115,4,1,False,False,0.977778,0.493096,0.702703,5.083378e+05,20135.135135
...,...,...,...,...,...,...,...,...,...,...
232713,5577726,296608,1,False,False,0.560000,0.409666,0.500000,1.876389e+08,12400.000000
232714,5577726,296609,0,False,False,0.573770,0.315236,0.500000,1.876389e+08,12400.000000
232715,5577726,296610,0,False,False,0.750000,0.523975,0.500000,1.876389e+08,12400.000000
232716,5577726,296611,0,False,False,0.541667,0.507859,0.500000,1.876389e+08,12400.000000
